In [21]:
from setup import osc, anh, morse, tw, columb, rel_osc, rel_morse, rel_tw, rel_columb, ur 

In [23]:
import torch
from samplers.Langevin import Langevin
N_samp=1000
z=torch.normal(mean=torch.zeros(N_samp,N_nod), std=torch.ones(N_samp,N_nod))
L=Langevin(ur,N_samp,eps=0.0001,N_sweep=1000000,log_per=1000)
L.run(z)

tensor(0.9998)
tensor(0.9271)
tensor(0.8675)
tensor(0.8216)
tensor(0.7829)
tensor(0.7497)
tensor(0.7218)
tensor(0.6950)
tensor(0.6732)
tensor(0.6574)
tensor(0.6407)
tensor(0.6297)
tensor(0.6193)
tensor(0.6055)
tensor(0.5938)
tensor(0.5878)
tensor(0.5776)
tensor(0.5705)
tensor(0.5612)
tensor(0.5509)
tensor(0.5473)
tensor(0.5433)
tensor(0.5373)
tensor(0.5339)
tensor(0.5295)
tensor(0.5265)
tensor(0.5223)
tensor(0.5187)
tensor(0.5195)
tensor(0.5167)
tensor(0.5155)
tensor(0.5145)
tensor(0.5134)
tensor(0.5098)
tensor(0.5092)
tensor(0.5053)
tensor(0.5027)
tensor(0.5034)
tensor(0.5058)
tensor(0.5054)
tensor(0.5057)
tensor(0.5040)
tensor(0.5031)
tensor(0.5043)
tensor(0.5053)
tensor(0.5073)
tensor(0.5064)
tensor(0.5070)
tensor(0.5040)
tensor(0.5035)
tensor(0.5039)
tensor(0.5059)
tensor(0.5090)
tensor(0.5098)
tensor(0.5128)
tensor(0.5154)
tensor(0.5157)
tensor(0.5154)
tensor(0.5129)
tensor(0.5145)
tensor(0.5154)
tensor(0.5184)
tensor(0.5217)
tensor(0.5184)
tensor(0.5201)
tensor(0.5185)
tensor(0.5

tensor([[ 0.2424, -0.1574, -0.6385,  ...,  0.5740,  0.2756,  0.1853],
        [-0.2886,  1.0603,  1.0730,  ...,  0.3106, -1.0518, -1.0839],
        [ 0.0917,  0.2289,  0.1406,  ..., -0.5484, -0.0724, -0.0565],
        ...,
        [-0.4469,  0.0263, -0.1147,  ...,  0.3881, -0.2816, -0.2089],
        [ 0.2520, -0.3598, -0.2020,  ...,  0.4637,  0.6337,  0.6345],
        [ 0.2099,  0.5716, -0.0290,  ..., -0.7093, -0.6123,  0.1158]])

In [19]:
import numpy as np
np.savetxt("trajs.txt",z.numpy())

In [11]:
from samplers.Metropolis import Metropolis
N_samp=100
z=torch.normal(mean=torch.zeros(N_samp,N_nod), std=torch.ones(N_samp,N_nod))
M=Metropolis(rel,N_samp,model="none",d=4/N_nod,N_sweep=10000,log_per=100)
M.run(z)

tensor(0.9743) 0
tensor(0.9062) tensor(0.6400)
tensor(0.8445) tensor(0.6100)
tensor(0.7795) tensor(0.5700)
tensor(0.7255) tensor(0.5300)
tensor(0.6837) tensor(0.5500)
tensor(0.6545) tensor(0.5700)
tensor(0.6221) tensor(0.5700)
tensor(0.5872) tensor(0.5300)
tensor(0.5580) tensor(0.5300)
tensor(0.5402) tensor(0.4700)
tensor(0.5150) tensor(0.5700)
tensor(0.5054) tensor(0.5300)
tensor(0.4852) tensor(0.4900)
tensor(0.4732) tensor(0.5700)
tensor(0.4607) tensor(0.5600)
tensor(0.4437) tensor(0.5000)
tensor(0.4305) tensor(0.6200)
tensor(0.4190) tensor(0.4700)
tensor(0.4117) tensor(0.4900)
tensor(0.4073) tensor(0.5100)
tensor(0.4086) tensor(0.5300)
tensor(0.4018) tensor(0.4800)
tensor(0.4036) tensor(0.5400)
tensor(0.4003) tensor(0.5500)
tensor(0.3964) tensor(0.4700)
tensor(0.3973) tensor(0.5100)
tensor(0.3904) tensor(0.3800)
tensor(0.3895) tensor(0.5200)
tensor(0.3852) tensor(0.5300)
tensor(0.3775) tensor(0.4500)
tensor(0.3710) tensor(0.5700)
tensor(0.3669) tensor(0.4900)
tensor(0.3687) tensor(0

tensor([[-0.2335,  0.0617,  0.3060,  ..., -0.7226, -0.0957, -0.0220],
        [-0.1943,  0.4442,  0.6643,  ...,  1.9976,  1.8974,  0.2973],
        [ 0.2043, -0.3873, -0.1802,  ..., -0.5318, -0.5652,  0.1541],
        ...,
        [ 0.2606, -0.1426, -0.1475,  ...,  1.1598,  1.2020,  0.4472],
        [-0.4772, -0.5529, -0.6534,  ..., -1.1712, -0.9370, -0.6515],
        [-0.7095, -0.4898,  0.3978,  ..., -0.4108, -0.3413, -0.6180]])